![hackio](https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_celeste@4x.png?raw=true)


# Laboratorio A/B Testing No Paramétricos

**Aerolínea:** 
Alas del Mundo

**Descripción de la Aerolínea**

Alas del Mundo es una aerolínea internacional que opera una amplia red de rutas a nivel mundial. La compañía se enorgullece de ofrecer vuelos cómodos y asequibles a una variedad de destinos, atrayendo a una diversa gama de clientes.

**Objetivo del Análisis**

El objetivo de este ejercicio es determinar si existe una diferencia significativa en el número de vuelos reservados entre pasajeros con diferentes niveles educativos y si existe una diferencia significativa entre el género de sus clientes y su salario. La aerolínea desea entender mejor a sus clientes para poder personalizar sus campañas de marketing y mejorar sus estrategias de ventas.

**Set de datos**

El conjunto de datos *alas_del_mundo.csv* contiene información sobre clientes de la aerolínea Alas del Mundo. A continuación, tenemos una descripción de lo que nos podemos encontrar en el conjunto de datos:

- **Loyalty Number**: Número de registro de cada uno de los clientes.

- **Province**: Provincia de residencia del cliente.

- **City**: Ciudad de residencia del cliente.

- **Gender**: Género con el que se identifica el cliente.

- **Education**: Nivel educativo del cliente.

- **Salary**: Salario anual del cliente.

- **Loyalty Card**: Nivel que tiene el cliente dentro del programa de fidelización de la aerolínea.

- **CLV**: Valor total del cliente para la aerolínea a lo largo de toda su relación.

- **Enrollment Type**: Forma de registro del cliente a la aerolínea.

- **Enrollment Year**: Año de inscripción.

- **Enrollment Month**: Mes de inscripción. 

- **Year**: Año en el que se realizan los vuelos.

- **Month**: Mes en el que se realizan los vuelos.

- **Flights Booked**: Vuelos reservados por el cliente en un mes y año concretos.

- **Distance**: Distancia de vuelo del cliente en un mes y año concreto

- **Points Accumulated**: Puntos acumulados por el cliente en un mes y año concretos.

## Ejercicio 1:

Antes de realizar el experimento A/B y formular hipótesis, es fundamental comprender a fondo el conjunto de datos y sus características. Para lograr esto, debes llevar a cabo un análisis exploratorio detallado de los datos (EDA), con el fin de familiarizarte con ellos y entender la información disponible.



In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

import matplotlib.pyplot as plt

from src import funciones as f

In [ ]:
aerolinea=pd.read_csv("datos/alas_del_mundo.csv")

In [28]:
aerolinea.head(3)

,Loyalty Number,Province,City,Gender,Education,Salary,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Year,Month,Flights Booked,Distance,Points Accumulated
0,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,1,0,0,0.0
1,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,2,3,2823,282.0
2,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,3,0,0,0.0


In [27]:
aerolinea.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403760 entries, 0 to 405623
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Loyalty Number      403760 non-null  int64  
 1   Province            403760 non-null  object 
 2   City                403760 non-null  object 
 3   Gender              403760 non-null  object 
 4   Education           403760 non-null  object 
 5   Salary              301500 non-null  float64
 6   Loyalty Card        403760 non-null  object 
 7   CLV                 403760 non-null  float64
 8   Enrollment Type     403760 non-null  object 
 9   Enrollment Year     403760 non-null  int64  
 10  Enrollment Month    403760 non-null  int64  
 11  Year                403760 non-null  int64  
 12  Month               403760 non-null  int64  
 13  Flights Booked      403760 non-null  int64  
 14  Distance            403760 non-null  int64  
 15  Points Accumulated  403760 non-null  fl

In [29]:
aerolinea[aerolinea["Salary"].isnull()].shape

(102260, 16)

In [25]:
aerolinea[aerolinea["Education"]== "College"].shape

(102260, 16)

In [46]:
aerolinea[aerolinea["Salary"] < 0].head(3)

,Loyalty Number,Province,City,Gender,Education,Salary,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Year,Month,Flights Booked,Distance,Points Accumulated
26328,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,1,0,0,0.0
26329,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,2,0,0,0.0
26330,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,3,0,0,0.0


Aqui paso a absolutos los negativos

In [47]:
aerolinea["Salary"]= aerolinea["Salary"].apply(lambda x: abs(x) if x < 0 else x)

In [48]:
aerolinea.describe().T

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,403760.0,549875.383713,258961.514684,100018.00,326699.00,550598.00,772152.00,999986.00
Salary,301500.0,79371.732902,34726.433958,9081.00,59262.00,73479.00,88606.00,407228.00
CLV,403760.0,7990.864857,6863.317160,1898.01,3981.78,5776.34,8937.12,83325.38
Enrollment Year,403760.0,2015.250716,1.979427,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,403760.0,6.667555,3.398829,1.00,4.00,7.00,10.00,12.00
Year,403760.0,2017.500352,0.500000,2017.00,2017.00,2018.00,2018.00,2018.00
Month,403760.0,6.501335,3.451982,1.00,4.00,7.00,10.00,12.00
Flights Booked,403760.0,4.134050,5.230064,0.00,0.00,1.00,8.00,21.00
Distance,403760.0,1214.460979,1434.098521,0.00,0.00,525.00,2342.00,6293.00
Points Accumulated,403760.0,124.263761,146.696179,0.00,0.00,53.00,240.00,676.50


In [40]:
aerolinea.describe(include=("O")).T

,count,unique,top,freq
Province,403760,11,Ontario,130258
City,403760,29,Toronto,80775
Gender,403760,2,Female,202757
Education,403760,5,Bachelor,252567
Loyalty Card,403760,3,Star,183745
Enrollment Type,403760,2,Standard,380419


In [8]:
aerolinea.duplicated().sum()

np.int64(1864)

In [21]:
aerolinea.drop_duplicates(inplace=True)

## EDA
La aerolinea cuenta con 16737 clientes en esta base de datos.

*CLV:*  
El valor del cliente para la aerolina tiene una media de 7990,86 lo que dista bastante de la mediana (5776,34) lo que muestra una distorsión de la media

*Enrollment Year y Month:* 
La media es el año 2015 y mes 6 lo que coincide con las medianas respectivas lo que tiene sentido.

*Flights Booked:*  
La media del número de vuelos cogidos por una persona es 4,13 lo que demuestra una elevada distorsión ya que la mediana es 1 vuelo. Además hemos visto como más del 25% de los clientes no han hecho ningun vuelo. Esto nos indica que la mayoría de clientes no reservan vuelos mientras que unos pocos si que son viajeros bastante habituales.

*Distancia y puntos acumulados:*  
Muestran de nuevo una gran distorsión entre la media y la mediana de la misma manera que habíamos identificado un sesgo en las reservas de vuelos.De hecho, tiene sentido que vayan de la mano.

*Salario:*  
Tras haber pasado a absolutos los negativos, ya podemos interpretar los valores.
la media de salario anual de los viajeros 79371,73 lo que no coincide por 6000 aproximadamente con la mediana lo que muestra un sesgo hacia los salarios más altos. Además la desviación estándar nos muestra una variabilidad extremadamente alta pues es de 34726 $ lo que nos muestra que hay clientes con salarios anuales extremadamente bajos como por ejemplo el mínimo de 9081 $ mientras que también hay clientes con salarios extramadamente altos como por ejemplo el máximo de 407228 $ (Grandes diferencias salariales)

 
**Nulos:**  
  
- He identificado más de 102260 nulos en el Salario y el total coincide con el total de clientes que van a College. Es por esto, que no podemos imputar ningún valor, eliminarlos ni inventarnos un dato por ello no queda otra que quedarnos con ellos.


**Duplicados:**    
- He observado que hay duplicados pero tras observarlos y analizarlos, decidí eliminarlos.


***Datos de Interés***  
  
- Hemos observado también que hay 480 salarios negativos que corresponden a gente en High School or below y Bachelor. Dado que nadie paga por trabajar, lo más probable es que se trate de un error tipográfico y por ello decidí pasarlo a absoluto. 
  




## Ejercicio 2:
Determina si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

 Pasos a seguir:

   - Definición de la hipótesis.
   - Prueba de hipótesis
   - Análisis de resultados
   - Conclusiones
   


## Ejercicio 3:
Determina si existe una diferencia significativa en el salario de los clientes en función del género de los mismos.

Pasos a seguir:
   - Definición de la hipótesis.
   - Prueba de hipótesis
   - Análisis de resultados
   - Conclusiones